# Detección de Idiomas en el Abstract

In [1]:
#Lectura de Librerías
import pandas as pd
import time

In [2]:
#Leyendo los abstracts
df = pd.read_csv(r'Data\scopus_con_indicadores.csv',sep=',')

C:\Users\YORLEY\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Remover Stopwords 

In [3]:
import nltk
from nltk import *
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\YORLEY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
stop = stopwords.words('english')

In [5]:
Abstracts = df[['Title','Abstract']].drop_duplicates()
Abstracts["Abstract"] = Abstracts["Abstract"].str.lower().str.split()
Abstracts.shape

(46837, 2)

In [6]:
start = time.time()
Abstracts['Clean_Abstract'] = Abstracts['Abstract'] \
.apply(lambda x: [item for item in x if item not in stop])
end = time.time()
print((end-start)/60)

0.2285290042559306


In [7]:
Abstracts['Clean_Abstract'] = Abstracts['Abstract'] \
.apply(lambda x: ' '.join([word for word in x]))

In [8]:
Abstracts[10:20]

,Title,Abstract,Clean_Abstract
35,C- And Sr-isotope stratigraphy of the São Caet...,"[c-isotope, and, 87sr/86sr, values, for, five,...",c-isotope and 87sr/86sr values for five carbon...
39,Morphosynthesis: High fidelity inorganic repli...,"[high, fidelity, calcium, carbonate, and, hydr...",high fidelity calcium carbonate and hydroxyapa...
41,A reassessment of the role of serotonergic sys...,"[the, role, of, serotonergic, system, in, the,...",the role of serotonergic system in the feeding...
46,Animal-based medicines: Biological prospection...,"[animals, have, been, used, as, medicinal, res...",animals have been used as medicinal resources ...
47,Determining sexual dimorphism in frog measurem...,"[several, analytic, techniques, have, been, us...",several analytic techniques have been used to ...
49,Mammalian cell invasion and intracellular traf...,"[trypanosoma, cruzi,, the, etiological, agent,...","trypanosoma cruzi, the etiological agent of ch..."
57,Mineral chemistry of tantalate species new in ...,"[tantalate, samples,, supposedly, of, the, col...","tantalate samples, supposedly of the columbite..."
62,Complementary Lagrangians in infinite dimensio...,"[we, prove, that, any, countable, family, of, ...",we prove that any countable family of lagrangi...
64,Chemical and mineralogical characterization of...,"[the, alto, quixaba, pegmatite,, seridó, regio...","the alto quixaba pegmatite, seridó region, nor..."
68,General optimal euclidean Sobolev and Gagliard...,"[we, prove, general, optimal, euclidean, sobol...",we prove general optimal euclidean sobolev and...


The common way of doing this is to transform the documents into tf-idf vectors, 
then compute the cosine similarity between them. Any textbook on information retrieval (IR) covers this. See esp. Introduction to Information Retrieval, which is free and available online.

Tf-idf (and similar text transformations) are implemented in the Python packages
Gensim and scikit-learn. In the latter package, computing cosine similarities is as easy as


In [9]:

from sklearn.feature_extraction.text import TfidfVectorizer


In [11]:
start=time.time()
abstracts = Abstracts['Clean_Abstract']
#stopwords_ = [word.decode('utf-8') for word in stopwords.words('english')]
tfidf = TfidfVectorizer().fit_transform(abstracts)
# no need to normalize, since Vectorizer will return normalized tf-idf
end = time.time()
print((end-start)/60)

0.11911589701970418


# La siguiente es la actividad que más recursos consume

In [ ]:
start=time.time()
pairwise_similarity = tfidf * tfidf.T
end = time.time()
print((end-start)/60)

In [16]:
pairwise_similarity.A

array([[1.        , 0.19398492, 0.16123592, 0.14078741, 0.11287005,
        0.1569428 , 0.19655586, 0.08490813, 0.16734285, 0.09044775],
       [0.19398492, 1.        , 0.22201381, 0.25308875, 0.15308845,
        0.24607412, 0.29720889, 0.20322919, 0.19696537, 0.1060546 ],
       [0.16123592, 0.22201381, 1.        , 0.1698412 , 0.10125079,
        0.17649501, 0.22828566, 0.10923815, 0.154202  , 0.1212646 ],
       [0.14078741, 0.25308875, 0.1698412 , 1.        , 0.18380305,
        0.23718104, 0.21522158, 0.17889136, 0.15634494, 0.10018152],
       [0.11287005, 0.15308845, 0.10125079, 0.18380305, 1.        ,
        0.14892747, 0.15749963, 0.11829638, 0.11773911, 0.08057711],
       [0.1569428 , 0.24607412, 0.17649501, 0.23718104, 0.14892747,
        1.        , 0.23921886, 0.12747713, 0.18373387, 0.09822092],
       [0.19655586, 0.29720889, 0.22828566, 0.21522158, 0.15749963,
        0.23921886, 1.        , 0.16066129, 0.215984  , 0.1122158 ],
       [0.08490813, 0.20322919, 0.1092381

### Otras posibles Opciones
https://stackoverflow.com/questions/51591510/text-similarity-approaches-do-not-reflect-real-similarity-between-texts
https://stackoverflow.com/questions/8897593/how-to-compute-the-similarity-between-two-text-documents
https://stackoverflow.com/questions/101569/algorithm-to-detect-similar-documents-in-python-script

#Apuntes
#https://stackoverflow.com/questions/25443802/unicode-warning-when-using-nltk-stopwords-with-tfidfvectorizer-of-scikit-learn

In [10]:
import numpy as np
from sklearn.cluster import SpectralClustering

In [17]:
mat= np.matrix(pairwise_similarity.A)

In [18]:
SpectralClustering(3).fit_predict(mat)

C:\Users\YORLEY\Anaconda3\lib\site-packages\sklearn\cluster\spectral.py:462: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"


array([0, 2, 2, 0, 0, 1, 0, 2, 0, 0, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 0, 2, 2, 2, 0, 0, 2, 2, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0,
       2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0,
       2, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2,
       2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2])